In [132]:
# ensure that any code changes are immediately reflected
%reload_ext autoreload
%autoreload 2

In [133]:
import pandas as pd
import numpy as np
import os
os.environ['MOSTLY_AI_PASSWORD'] = 'Mostly01!'

df = pd.DataFrame({'name': np.random.choice(['Alice', 'Bob', 'Charlie', 'David', 'Eva'], 10_000), 
                   'age': np.random.randint(18, 65, size=10_000), 
                   'income': np.random.normal(50000, 15000, 10_000)})
df

,name,age,income
0,David,22,38387.090177
1,Bob,25,56014.069146
2,Alice,58,51098.423161
3,Eva,22,44830.592933
4,Eva,59,18504.763659
...,...,...,...
9995,Alice,47,77796.322951
9996,Charlie,45,43990.762523
9997,Eva,35,52121.722088
9998,Charlie,64,60466.414154


In [134]:
from mostlyai import MostlyAI
client = MostlyAI(
    # api_key = 'xxx',         # or MOSTLYAI_API_KEY env var; this is bound to a single user account
    # base_url = 'https://..'  # optional; defaults to our free version
)

In [135]:
g = client.generators.create(tables=[{'data': df, 'name': 'michi'}], start=True, wait=True)

Job status: 18it [00:35,  1.99s/it]                      

Job finished in 36.0s


In [136]:
sd = client.synthetic_datasets.create(generatorId=g.id, start=True, wait=True)

Job status: 5it [00:41,  8.21s/it]                       

Job finished in 41.2s


In [137]:
dfs = sd.data()
dfs

APIStatusError: HTTP error occurred: 400 b'{"message":"Unable to download the synthetic dataset as the job associated with ID &#39;90fe9f16-3a75-4a51-b5f1-b830532f6a7e&#39; is not in &#39;DONE&#39; status. Dataset downloads are only available for completed jobs"}'